In [1]:
import torch
print(torch.cuda.is_available())  # Should print: True
print(torch.cuda.get_device_name(0))  # Your GPU name


True
NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch
import os

c:\final\finetune\fineenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Load model in 4-bit (QLoRA)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [5]:
from datasets import load_dataset

# If your file is named 'subset.json' and in the same directory
dataset = load_dataset("json", data_files="scientific_summarization_chat.json")

Generating train split: 2500 examples [00:00, 71043.66 examples/s]


In [7]:
from transformers import AutoTokenizer
from peft import prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from trl import SFTTrainer

In [21]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Required for Trainer

In [25]:
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,  # Faster
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)
model.enable_input_require_grads()

In [26]:
# Format dataset into strings
def format_chat(example):
    messages = example["messages"]
    formatted = ""
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            formatted += f"<|user|>\n{content}\n"
        elif role == "assistant":
            formatted += f"<|assistant|>\n{content}\n"
        elif role == "system":
            formatted += f"<|system|>\n{content}\n"
    return {"text": formatted.strip()}

tokenized_dataset = dataset["train"].map(format_chat)

In [29]:



training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    # Remove or comment out the following line if your transformers version is old:
    # evaluation_strategy="no",
    fp16=True,
)


trainer.train()

AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.

In [11]:
# LoRA config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # standard for LLaMA models
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [12]:
model = get_peft_model(model, peft_config)

In [14]:
# Load dataset
dataset = load_dataset("json", data_files="scientific_summarization_chat.json")

In [15]:
# Format data
def format_chat(example):
    prompt = tokenizer.apply_chat_template(example["messages"], tokenize=False, add_generation_prompt=True)
    return {"input_ids": tokenizer(prompt, truncation=True, padding="max_length", max_length=512)["input_ids"],
            "labels": tokenizer(prompt, truncation=True, padding="max_length", max_length=512)["input_ids"]}

In [16]:
tokenized_dataset = dataset["train"].map(format_chat)

Map: 100%|██████████| 2500/2500 [00:11<00:00, 215.25 examples/s]


In [ ]:
# Training config
training_args = TrainingArguments(
    output_dir="./qlora-finetuned-tinyllama",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    bf16=False,
    fp16=True,
    report_to="none"
)

In [18]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [20]:
# Start training
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\final\finetune\fineenv\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\final\finetune\fineenv\Lib\site-packages\bitsandbytes\nn\modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\final\finetune\fineenv\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In versi

Step,Training Loss


KeyboardInterrupt: 